## Importing Libraries

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os


In [17]:
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "D:\DigiMark\Exracting Word Via Images\src\data\in1.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
}

payload = {
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "system",
      "content": "You are an experienced mobile developer, a web scraper, and an expert in image processing and generative AI. You will be provided with an image, which could be a screenshot of a document or PDF, or it could contain handwritten text. Your task is to extract all of the relevant data from the image and return it in a specified format."
    },
    {
      "role": "user",
      "content": [{  
                    "type":"text",
                    "text":"please give me whatever text is in the image"
                  },
                   {
                      "type": "image_url",
                      "image_url": {
                      "url": f"data:image/jpg;base64,{base64_image}"
                      }                     
                   }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9FiFNHR2fxHkaNyufqQrPCtRh8ivT', 'object': 'chat.completion', 'created': 1713532033, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '**Product Purchase**\nDate: Apr 12, 2024\n\n**Vendor Details:**\nJS Auto Suppliers\n5678 Oak Avenue\nPortland, OR 97201\n(503) 555-6789\ninfo@jsautosuppliers.com\n\n**Business Details:**\nAuto Parts Store\n1234 Maple Street\nSpringfield, IL 62701\n(217) 555-1234\ninfo@autopartsstore.com\n\n**Items Purchased:**\n\n| Item                  | Price (USD) | Qty | Add Tax | Total (USD) |\n|-----------------------|-------------|-----|---------|-------------|\n| Engine Oil (5W-30)    | 25.00       | 10  | 8%      | 220.00      |\n| Brake Pads (Front)    | 50.00       | 5   | 8%      | 215.00      |\n| Air Filter (Toyota Camry) | 15.00  | 20  | 8%      | 108.00      |\n| Spark Plugs (Set of 4)| 20.00       | 8   | 8%      | 129.60      |\n| Transmission Fluid (1 qt) | 10.00   | 15  | 8%      

In [6]:
import json

# JSON response string
json_response = "{\n\t\"last_balance_sheet_date\": \"January 31, 2024\",\n\t\"cash_on_hand\": \"$10,000\",\n\t\"bank_name\": \"First National Bank\",\n\t\"bank_balance\": \"$50,000\",\n\t\"list_of_products\": [\n\t\t{\n\t\t\t\"product_name\": \"Widget A\",\n\t\t\t\"product_cost\": \"$10\",\n\t\t\t\"quantity\": \"100\",\n\t\t\t\"delivery_charges\": \"$5\"\n\t\t},\n\t\t{\n\t\t\t\"product_name\": \"Widget B\",\n\t\t\t\"product_cost\": \"$20\",\n\t\t\t\"quantity\": \"50\",\n\t\t\t\"delivery_charges\": \"$8\"\n\t\t},\n\t\t{\n\t\t\t\"product_name\": \"Widget C\",\n\t\t\t\"product_cost\": \"$15\",\n\t\t\t\"quantity\": \"75\",\n\t\t\t\"delivery_charges\": \"$6\"\n\t\t}\n\t]\n}"

# Parse JSON string
data = json.loads(json_response)

# Write formatted JSON data to a file
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)

In [10]:

import os
import config
from utils import encode_image, extract_file_type
import requests
from dotenv import load_dotenv
import json
load_dotenv()


file_path = 'data/bank_sheet.png'


base64_image = encode_image(file_path)

headers = {
"Content-Type": "application/json",
"Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
}

payload = {
    'model':"gpt-4-turbo",
    'messages': config.PROMPT + [{
    "role": "user",
    "content": [{
        "type": "image_url",
        "image_url": {
            "url": f"data:image/" + extract_file_type(file_path) + f";base64,{base64_image}"
        }
    }]
}],
    'max_tokens':1000,
}
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

with open('../output.json', 'w') as f:
    json.dump(response.json()['choices'][0]['message']['content'], f, indent=4)


In [9]:
import json

json_response = response.json()['choices'][0]['message']['content']

data = json.loads(json_response)

with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)


In [ ]:
"""
What: Please help me extract the textual pieces of information from the image provided. The image you will receive will have data in tabular format, your job is to focus on the tabular data and ignore everything else.The desired output I want to receive is in JSON format and just this nothing else.

Who: I am a professional Mobile application developer in Flutter.

Why: This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.

Which: Please observe the column names of the table as they are supposed to be key names in the JSON also populate each key with data provided with respect to each column.The data you are supposed to extract from the image is 'Item', 'Price(USD), 'Quantity', 'Add Tax', 'Total (USD)'. Please note that the wording of the column names may differ, please  extract them according to the context.Following is the template on how you should create the JSON:
{
{'Item':, 
'Price(USD):, 
'Quantity', 
'Add Tax',
 'Total (USD)
}
}

How: Finally, return the data as a JSON response and only JSON nothing else.
"""
